In [5]:
from numpy import pi

1e11 * (1.0 - 1.0 / (1.0 + (2.0*pi/(86400*365.25*100)) / (1e11/3e20) * 1.0j)) / 1e11

(0.9727353219191213+0.16285366256251269j)

In [1]:
from utils import (
    RealDescription,
    ElasticityDescription,
    LoveNumbersHyperParameters,
    load_base_model,
    parameters_path,
    Earth_radius,
    elastic_Love_numbers_computing,
    elasticity_descriptions_path,
)
from numpy import ones

Love_numbers_hyper_parameters: LoveNumbersHyperParameters = load_base_model(
    name="Love_numbers_hyper_parameters", path=parameters_path, base_model_type=LoveNumbersHyperParameters
)
Love_numbers_hyper_parameters.load()

# Generates elasticity description.
elasticity_description = ElasticityDescription(
    radius_unit=(
        Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        if Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        else Earth_radius
    ),
    below_ICB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_ICB_layers,
    real_crust=Love_numbers_hyper_parameters.real_description_parameters.real_crust,
    n_splines_base=10,
    profile_precision=10,
    below_CMB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_CMB_layers,
    splines_degree=Love_numbers_hyper_parameters.real_description_parameters.splines_degree,
    model_filename="PREM",
    id="mu11",
    load_description=False,
)

# Force mu value.
forced_mu = 1e11 / (
    elasticity_description.density_unit * elasticity_description.radius_unit**2 / elasticity_description.period_unit**2
)
for i_layer, layer in enumerate(elasticity_description.description_layers):
    elasticity_description.description_layers[i_layer].splines["mu_0"] = (
        layer.splines["mu_0"][0],
        forced_mu * ones(shape=layer.splines["mu_0"][1].shape),
        layer.splines["mu_0"][2],
    )
# Saves new description.
elasticity_description.save(path=elasticity_descriptions_path)

# Generates real description using new elasticity description.
real_description = RealDescription(
    id="test-mu11",
    below_ICB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_ICB_layers,
    below_CMB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_CMB_layers,
    splines_degree=Love_numbers_hyper_parameters.real_description_parameters.splines_degree,
    radius_unit=(
        Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        if Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        else Earth_radius
    ),
    real_crust=Love_numbers_hyper_parameters.real_description_parameters.real_crust,
    n_splines_base=10,
    profile_precision=10,
    radius=(
        Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        if Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        else Earth_radius
    ),
    load_description=False,
    elasticity_description_from_id="mu11"
)

In [2]:
from utils import Result

# Verifies elastic results.
degrees = [1, 2]
elastic_Love_numbers = elastic_Love_numbers_computing(
    y_system_hyper_parameters=Love_numbers_hyper_parameters.y_system_hyper_parameters,
    degrees=degrees,
    real_description=real_description,
)

elastic_result = Result(hyper_parameters=Love_numbers_hyper_parameters)
elastic_result.update_values_from_array(result_array=elastic_Love_numbers, degrees=degrees)
elastic_result.values

{<Direction.radial: 0>: {<BoundaryCondition.load: 0>: array([[-0.30580461+0.j],
         [-1.29668031+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.16984415+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.90243503+0.j]])},
 <Direction.tangential: 1>: {<BoundaryCondition.load: 0>: array([[ 0.09162569+0.j],
         [-0.05537723+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.72094023+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.28431107+0.j]])},
 <Direction.potential: 2>: {<BoundaryCondition.load: 0>: array([[ 0.        +0.j],
         [-0.90881643+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.28431107+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.89605362+0.j]])}}